In [1]:
# This file is used to split the segmented videos into clips.!!!!!!!!!!!!!!!!!!
# Also last code is used to convert the clips into train-test-validation split




import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling3D
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers.core import Activation
from keras.layers import Input, Dense, Flatten, Dropout, Conv3D, MaxPooling3D, BatchNormalization,AveragePooling3D , concatenate


2023-09-16 16:46:07.060700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
video_paths =['Converted/10.mp4','Converted/20.mp4','Converted/30.mp4','Converted/40.mp4','Converted/50.mp4','Converted/60.mp4','Converted/70.mp4']
video_path=video_paths[2]

In [27]:
import cv2
import numpy as np

cap = cv2.VideoCapture(video_path)

# Get video properties from the original video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_file = 'Optical_flow/OP_30_n.mp4'
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Use 'H264' for MP4 format
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Initialize variables
prev_gray = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_gray is not None:
        # Calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale=0.5, levels=3, winsize=15,
                                            iterations=3, poly_n=5, poly_sigma=1.2,
                                            flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

        # Calculate magnitude of optical flow vectors
        magnitude = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)

        # Highlight the regions of optical flow velocity
        threshold = 1.5
        flow_threshold = np.where(magnitude > threshold, 255, 0).astype(np.uint8)

        contours, _ = cv2.findContours(flow_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        farme_with_border = frame.copy()
        cv2.drawContours(farme_with_border ,contours, -1,(0,0,255),2)#red border with border size 2
        #flow_bgr = cv2.merge((flow_threshold, flow_threshold, flow_threshold))

        #output_frame = cv2.addWeighted(frame, 0.7, flow_bgr, 0.3, 0)

        cv2.imshow('Optical Flow Visualization', farme_with_border)

        out.write(farme_with_border)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    prev_gray = gray

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture(video_path)

# Get video properties from the original video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_file = 'Optical_flow/OP_50.mp4'
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Use 'H264' for MP4 format
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Initialize variables
prev_gray = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_gray is not None:
        # Calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale=0.5, levels=5, winsize=15,
                                            iterations=5, poly_n=5, poly_sigma=1.2,
                                            flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

        # Calculate magnitude of optical flow vectors
        magnitude = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)

        # Highlight the regions of optical flow velocity
        threshold = 1.6
        flow_threshold = np.where(magnitude > threshold, 255, 0).astype(np.uint8)

        contours, _ = cv2.findContours(flow_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        mask = np.zeros_like(frame)
        
        cv2.drawContours(mask, contours, -1,(255,0,255), thickness=cv2.FILLED)
        
        object_inside_contour = cv2.bitwise_and(frame,mask)
        
        combined_frame = np.hstack((frame, object_inside_contour))
        
        average_velocity = np.mean(magnitude[np.where(magnitude>threshold)])
        
        cv2.putText(combined_frame, f'Average Velocity : {average_velocity:.3f}',
                   (1316,30),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2,cv2.LINE_AA)
        
        cv2.imshow('Optical Flow Visualization', cv2.resize(combined_frame,(896,448)))

        #out.write(object_inside_contour)
        

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    prev_gray = gray

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


In [ ]:
import os
import glob
import torch
import torchvision.io
from sklearn.model_selection import train_test_split

# Define the directory containing the dataset
data_dir = 'Optical_Flow_Threshold3/'

# Define the directory to store the clips
output_dir = 'Klips/'

# Define the list of classes (folder names) in the dataset
classes = ['10', '20', '30', '40', '50', '60', '70']

# Function to create directories if they don't exist
def create_directories():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for class_name in classes:
        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

# Function to load video clips and their labels from the dataset directory
def load_video_clips(data_dir, classes, num_frames=16, stride=16):
    video_clips = []
    labels = []

    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        video_files = glob.glob(os.path.join(class_dir, '*.mp4'))

        for video_file in video_files:
            # Read video frames
            frames, _, _ = torchvision.io.read_video(video_file)
            num_frames_video = frames.shape[0]

            # Create video clips with the specified stride
            for i, start in enumerate(range(0, num_frames_video - num_frames + 1, stride)):
                end = start + num_frames
                clip = frames[start:end]
                video_clips.append(clip)
                labels.append(int(class_name))

                # Save the clip to the output directory
                output_class_dir = os.path.join(output_dir, class_name)
                output_file = os.path.join(output_class_dir, f'clip{i + 1}.mp4')  # You can replace '.xyz' with the appropriate video format
                fps = 30
                torchvision.io.write_video(output_file, clip, fps = fps)

    return video_clips, labels

# Create the output directories
create_directories()

# Load video clips and their labels
video_clips, labels = load_video_clips(data_dir, classes)


/home/server_admin/anaconda3/envs/soumya_2002/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [3]:
import os
import glob
import torch
import torchvision.io
from sklearn.model_selection import train_test_split

# Define the directory containing the dataset
data_dir = 'Optical_Flow_Threshold_3/'

# Define the directory to store the clips
output_dir = 'Llip/'

# Define the list of classes (folder names) in the dataset
classes = ['10', '20', '30', '40', '50', '60', '70']

# Function to create directories if they don't exist
def create_directories():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for class_name in classes:
        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

# Function to load video clips and their labels from the dataset directory
def load_video_clips(data_dir, classes, num_frames=16, stride=8):
    video_clips = []
    labels = []

    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        video_files = glob.glob(os.path.join(class_dir, '*.mp4'))

        for video_file in video_files:
            # Read video frames
            frames, _, _ = torchvision.io.read_video(video_file)
            num_frames_video = frames.shape[0]

            # Create video clips with the specified stride
            for i, start in enumerate(range(0, num_frames_video - num_frames + 1, stride)):
                end = start + num_frames
                clip = frames[start:end]
                video_clips.append(clip)
                labels.append(int(class_name))

                # Save the clip to the output directory
                output_class_dir = os.path.join(output_dir, class_name)
                output_file = os.path.join(output_class_dir, f'clip{i + 1+ 1069}.mp4')  # You can replace '.xyz' with the appropriate video format
                fps = 5
                torchvision.io.write_video(output_file, clip, fps=fps)

            # Clear the lists to free up memory after processing each class
            video_clips.clear()
            labels.clear()

    return video_clips, labels

# Create the output directories
create_directories()

# Load video clips and their labels
video_clips, labels = load_video_clips(data_dir, classes)

In [18]:
import os
import glob
import torch
import torchvision.io
from sklearn.model_selection import train_test_split

# Define the directory containing the dataset
data_dir = 'Unseen_Data/'

# Define the directory to store the clips
output_dir = 'Unseen/'

# Define the list of classes (folder names) in the dataset
classes = ['10', '20', '30', '40', '50', '60', '70']

# Function to create directories if they don't exist
def create_directories():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for class_name in classes:
        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

# Function to load video clips and their labels from the dataset directory
def load_video_clips(data_dir, classes, num_frames=16, stride=8):
    video_clips = []
    labels = []

    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        video_files = glob.glob(os.path.join(class_dir, '*.mp4'))

        for video_file in video_files:
            # Read video frames
            frames, _, _ = torchvision.io.read_video(video_file)
            num_frames_video = frames.shape[0]

            # Create video clips with the specified stride
            for i, start in enumerate(range(0, num_frames_video - num_frames + 1, stride)):
                end = start + num_frames
                clip = frames[start:end]
                video_clips.append(clip)
                labels.append(int(class_name))

                # Save the clip to the output directory
                output_class_dir = os.path.join(output_dir, class_name)
                output_file = os.path.join(output_class_dir, f'clip{i +1}.mp4')  # You can replace '.xyz' with the appropriate video format
                fps = 5
                torchvision.io.write_video(output_file, clip,fps=fps)

            # Clear the lists to free up memory after processing each class
            video_clips.clear()
            labels.clear()

    return video_clips, labels

# Create the output directories
create_directories()

# Load video clips and their labels
video_clips, labels = load_video_clips(data_dir, classes)

In [5]:
import os
import random
from sklearn.model_selection import train_test_split

# Define the directory containing the clips
klips_dir = 'Llips/'

# Define the list of classes (class folder names)
classes = ['10', '20', '30', '40', '50', '60', '70']

# Function to split clips into train, validation, and test sets for each class
def split_clips_by_class(class_dir, train_size=0.7, test_size=0.15, validation_size=0.15, random_seed=None):
    clip_files = os.listdir(class_dir)
    
    # Split clips into train, validation, and test sets
    train_clips, temp_clips = train_test_split(clip_files, train_size=train_size, random_state=random_seed)
    validation_test_clips = train_test_split(temp_clips, test_size=test_size / (1 - train_size), random_state=random_seed)
    validation_clips, test_clips = validation_test_clips[0], validation_test_clips[1]
    
    return train_clips, validation_clips, test_clips

# Function to move clips to their respective subdirectories
def move_clips_to_split(class_dir, split_dir, clip_files):
    for clip_file in clip_files:
        src_path = os.path.join(class_dir, clip_file)
        dst_path = os.path.join(split_dir, clip_file)
        os.rename(src_path, dst_path)

# Create 'Train', 'Validation', and 'Test' directories
train_dir = os.path.join(klips_dir, 'Train')
os.makedirs(train_dir, exist_ok=True)

validation_dir = os.path.join(klips_dir, 'Validation')
os.makedirs(validation_dir, exist_ok=True)

test_dir = os.path.join(klips_dir, 'Test')
os.makedirs(test_dir, exist_ok=True)

# Iterate through each class folder and split clips
for class_name in classes:
    class_dir = os.path.join(klips_dir, class_name)
    
    # Create class-specific subdirectories in 'Train', 'Validation', and 'Test'
    class_train_dir = os.path.join(train_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    
    class_validation_dir = os.path.join(validation_dir, class_name)
    os.makedirs(class_validation_dir, exist_ok=True)
    
    class_test_dir = os.path.join(test_dir, class_name)
    os.makedirs(class_test_dir, exist_ok=True)
    
    # Split clips for the current class
    clip_files = os.listdir(class_dir)
    train_clips, validation_clips, test_clips = split_clips_by_class(class_dir, train_size=0.7, test_size=0.15, validation_size=0.15, random_seed=42)
    
    # Move clips to their respective subdirectories
    move_clips_to_split(class_dir, class_train_dir, train_clips)
    move_clips_to_split(class_dir, class_validation_dir, validation_clips)
    move_clips_to_split(class_dir, class_test_dir, test_clips)